## 1. Load the dataset

[뉴욕 승차공유 플랫폼 탑승 데이터](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_hvfhs.pdf)

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator, StrMethodFormatter

df = pd.read_csv('./data/fhvhv_tripdata_2023.csv')

In [ ]:
pd.set_option('float_format', '{:.2f}'.format)

df.info() # 전체 데이터 정보와 결측치 확인 가능

## 2. Data description

In [ ]:
df.describe() # 데이터의 기초 통계량 확인 가능

In [ ]:
# hvfhs_license_num: HVFHS; HV0003: Uber, HV0005: Lyft
# dispatching_base_num: B03404: Uber, B02835: Lyft
# originating_dispatch_base_num: B03404: Uber, B02835: Lyft
# requested_pickup_time: 요청한 픽업 시간
# on_scene_datetime: 픽업 장소에 도착한 시간
# pickup_datetime: 승차한 시간
# dropoff_datetime: 하차한 시간
# passenger_count: 승객 수
# PULocationID: 승차 위치 ID
# DOLocationID: 하차 위치 ID
# trip_miles: 여행한 거리
# trip_times: 여행한 시간
# base_passenger_fare: 기본 승객 요금
# tolls: 통행료
# bcf: black car fund
# sales_tax: 판매세
# congestion_surcharge: 혼잡료
# airport_fee: 공항 요금
# tips: 팁
# driver_pay: 드라이버의 총 지급액
# shared_request_flag: 승객이 공유 픽업에 동의했는지 여부
# shared_match_flag: 승객이 다른 승객과 공유되었는지 여부
# access_a_ride_flag: MTA를 대신해 승객을 태웠는지 여부
# wav_request_flag: 승객이 휠체어 접근 가능한 차량을 요청했는지 여부
# wav_matched_flag: 승객이 휠체어 접근 가능한 차량을 탔는지 여부

In [ ]:
df

## 3. Missing value check

In [ ]:
missing_values = df.isnull().sum()
missing_values

In [ ]:
# 결측치 대체
df['on_scene_datetime'].fillna('Unknown', inplace=True)

# 결측치 제거
df.dropna(subset=['on_scene_datetime'], inplace=True)

In [ ]:
missing_values = df.isnull().sum()
missing_values

## 4. Outlier check

In [ ]:
# pickup_datetime을 datetime 형식으로 변환
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['month'] = df['pickup_datetime'].dt.to_period('M')

In [ ]:
# Seaborn 스타일 설정
sns.set(style="whitegrid")

# hvfhs_license_num별로 데이터를 나누어 각각 Boxplot을 그림
license_nums = ['HV0003', 'HV0005']

for license_num in license_nums:
    plt.figure(figsize=(14, 7))
    sns.boxplot(x='month', y='trip_miles', data=df[df['hvfhs_license_num'] == license_num])
    plt.title(f'Trip Miles Monthly Boxplot for {license_num}')
    plt.xticks(rotation=45)
    plt.xlabel('Month')
    plt.ylabel('Trip Miles')
    plt.show()

## 5. Data visualization

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 월별로 hvfhs_license_num의 빈도수 계산
df['month'] = pd.to_datetime(df['pickup_datetime']).dt.month
monthly_counts = df.groupby(['month', 'hvfhs_license_num']).size().reset_index(name='counts')

# 'month' 열을 문자열로 변환
monthly_counts['month'] = monthly_counts['month'].apply(lambda x: f'Month {x}')

# 단일 축 그래프를 사용하여 월별 빈도를 함께 표시
plt.figure(figsize=(12, 6))
sns.barplot(data=monthly_counts, x='month', y='counts', hue='hvfhs_license_num')

# 평균값에 대한 선 추가
average_counts = monthly_counts.groupby('hvfhs_license_num')['counts'].mean().reset_index()
for index, row in average_counts.iterrows():
    plt.axhline(y=row['counts'], color='red', linestyle='--')

plt.legend(title='HVFHS License Number', loc='upper left', bbox_to_anchor=(1, 1))

plt.xlabel('Month')
plt.ylabel('Frequency')
plt.title('Monthly Frequency of HVFHS License Numbers')
plt.xticks(rotation=45)  # x축 라벨 회전
plt.show()

In [ ]:
import matplotlib.pyplot as plt

df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['trip_time_min'] = df['trip_time'] / 60
numeric_cols = ['trip_miles', 'trip_time_min', 'driver_pay']
grouped = df[numeric_cols].groupby(df['pickup_datetime'].dt.hour).mean()

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(grouped['trip_miles'], marker='o')
plt.title('Average Trip Miles by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Average Trip Miles')

plt.subplot(1, 3, 2)
plt.plot(grouped['trip_time_min'], marker='o', color='orange')
plt.title('Average Trip Time by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Average Trip Time (min)')

plt.subplot(1, 3, 3)
plt.plot(grouped['driver_pay'], marker='o', color='green')
plt.title('Average Driver Pay by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Average Driver Pay')

plt.show()

## 6. Modeling

In [ ]:
import pandas as pd

def assign_time_category(hour):
    if hour < 6:
        return 'dawn'
    elif 6 <= hour < 10:
        return 'morning'
    elif 10 <= hour < 16:
        return 'afternoon'
    elif 16 <= hour < 20:
        return 'evening'
    else:
        return 'night'

def filter_and_group_by_time_and_license(df, license_num):
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df['time_category'] = df['pickup_datetime'].dt.hour.apply(assign_time_category)

    filtered_grouped = df[df['hvfhs_license_num'] == license_num].groupby('time_category')
    return filtered_grouped

# Example usage
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime']) # Assuming df is your DataFrame
uber_groups = filter_and_group_by_time_and_license(df, 'HV0003')
lyft_groups = filter_and_group_by_time_and_license(df, 'HV0005')

In [ ]:
plt.figure(figsize=(15, 30))

sorted_time_categories = ['dawn', 'morning', 'afternoon', 'evening', 'night']
uber_avg_trip_miles = uber_groups['trip_miles'].mean().reindex(sorted_time_categories)
uber_avg_trip_time = uber_groups['trip_time'].mean().reindex(sorted_time_categories)
uber_avg_fare = uber_groups['base_passenger_fare'].mean().reindex(sorted_time_categories)
uber_driver_pay = uber_groups['driver_pay'].mean().reindex(sorted_time_categories)

lyft_avg_trip_miles = lyft_groups['trip_miles'].mean().reindex(sorted_time_categories)
lyft_avg_trip_time = lyft_groups['trip_time'].mean().reindex(sorted_time_categories)
lyft_avg_fare = lyft_groups['base_passenger_fare'].mean().reindex(sorted_time_categories)
lyft_driver_pay = lyft_groups['driver_pay'].mean().reindex(sorted_time_categories)

plt.subplot(4, 1, 1)
plt.plot(uber_avg_trip_miles, marker='o', color='purple')
plt.plot(lyft_avg_trip_miles, marker='o', color='orange')
plt.title('Average Trip Miles by Time Category')

plt.subplot(4, 1, 2)
plt.plot(uber_avg_trip_time, marker='o', color='purple')
plt.plot(lyft_avg_trip_time, marker='o', color='orange')
plt.title('Average Trip Time by Time Category')

plt.subplot(4, 1, 3)
plt.plot(uber_avg_fare, marker='o', color='purple')
plt.plot(lyft_avg_fare, marker='o', color='orange')
plt.title('Average Fare by Time Category')

plt.subplot(4, 1, 4)
plt.plot(uber_driver_pay, marker='o', color='purple')
plt.plot(lyft_driver_pay, marker='o', color='orange')
plt.title('Average Driver Pay by Time Category')

plt.legend(['Uber', 'Lyft'], loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
uber_avg_trip_miles, uber_avg_trip_time, uber_avg_fare, uber_driver_pay

In [ ]:
lyft_avg_trip_miles, lyft_avg_trip_time, lyft_avg_fare, lyft_driver_pay

In [ ]:
df['time_category'] = df['pickup_datetime'].dt.hour.apply(assign_time_category)

# 시간대 카테고리를 Categorical 타입으로 변환하고 정렬
sorted_time_categories = ['dawn', 'morning', 'afternoon', 'evening', 'night']
df['time_category'] = pd.Categorical(df['time_category'], categories=sorted_time_categories, ordered=True)

In [ ]:
df

In [ ]:
# hvfhs_license_num: HVFHS; HV0003: Uber, HV0005: Lyft
# dispatching_base_num: B03404: Uber, B02835: Lyft
# originating_dispatch_base_num: B03404: Uber, B02835: Lyft
# requested_pickup_time: 요청한 픽업 시간
# on_scene_datetime: 픽업 장소에 도착한 시간
# pickup_datetime: 승차한 시간
# dropoff_datetime: 하차한 시간
# passenger_count: 승객 수
# PULocationID: 승차 위치 ID
# DOLocationID: 하차 위치 ID
# trip_miles: 여행한 거리
# trip_times: 여행한 시간
# base_passenger_fare: 기본 승객 요금
# tolls: 통행료
# bcf: black car fund
# sales_tax: 판매세
# congestion_surcharge: 혼잡료
# airport_fee: 공항 요금
# tips: 팁
# driver_pay: 드라이버의 총 지급액
# shared_request_flag: 승객이 공유 픽업에 동의했는지 여부
# shared_match_flag: 승객이 다른 승객과 공유되었는지 여부
# access_a_ride_flag: MTA를 대신해 승객을 태웠는지 여부
# wav_request_flag: 승객이 휠체어 접근 가능한 차량을 요청했는지 여부
# wav_matched_flag: 승객이 휠체어 접근 가능한 차량을 탔는지 여부

In [ ]:
features = ['PULocationID', 'DOLocationID', 'trip_miles', 'trip_time', 'base_passenger_fare', 
            'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'time_category']

X = df[features]
y = df['driver_pay']

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 수치형 및 범주형 특성 구분
numeric_features = ['trip_miles', 'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips']
categorical_features = ['PULocationID', 'DOLocationID', 'time_category']

# 전처리 파이프라인 설정
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
# 전체 파이프라인 설정 (전처리 + 모델)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# 모델 훈련
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')